In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime

In [3]:
# LOAD DATA

print("Loading transit performance data...")
detailed_df = pd.read_csv('transit_performance_detailed.csv')
print(f"Loaded {len(detailed_df)} shipment records")
print()


Loading transit performance data...
Loaded 99 shipment records



In [4]:
# 1. DATA CLEANING FOR VISUALIZATION

print("Cleaning data for analysis...")
# Drop rows where total_transit_hours is NaN or zero
df_clean = detailed_df.dropna(subset=['total_transit_hours']).copy()
df_clean = df_clean[df_clean['total_transit_hours'] > 0]

print(f"After cleaning: {len(df_clean)} shipments")
print(f"Removed {len(detailed_df) - len(df_clean)} incomplete records")
print()

Cleaning data for analysis...
After cleaning: 99 shipments
Removed 0 incomplete records



In [5]:
# 2. CREATE VISUALIZATIONS

# Set consistent style
sns.set_style("whitegrid")
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 7)
plt.rcParams['font.size'] = 10

print("Generating visualizations...")
print("-" * 80)

Generating visualizations...
--------------------------------------------------------------------------------


In [6]:
# --- 2.1. Distribution of Total Transit Time ---
print("Creating: Transit Time Distribution...")
plt.figure(figsize=(12, 7))
sns.histplot(df_clean['total_transit_hours'], bins=25, kde=True, color='#3498db', edgecolor='black')
plt.axvline(df_clean['total_transit_hours'].mean(), color='red', linestyle='--', 
            linewidth=2, label=f'Mean: {df_clean["total_transit_hours"].mean():.1f}h')
plt.axvline(df_clean['total_transit_hours'].median(), color='green', linestyle='--', 
            linewidth=2, label=f'Median: {df_clean["total_transit_hours"].median():.1f}h')
plt.title('Distribution of Total Transit Time (Hours)', fontsize=16, fontweight='bold')
plt.xlabel('Total Transit Hours', fontsize=12)
plt.ylabel('Number of Shipments', fontsize=12)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('transit_time_distribution.png', dpi=300, bbox_inches='tight')
plt.close()
print("✓ Saved: transit_time_distribution.png")


Creating: Transit Time Distribution...
✓ Saved: transit_time_distribution.png


In [7]:
# --- 2.2. Average Transit Hours by Origin State ---
print("Creating: Transit Hours by Origin State...")
state_performance = df_clean.groupby('origin_state').agg({
    'total_transit_hours': 'mean',
    'tracking_number': 'count'
}).reset_index()
state_performance.columns = ['origin_state', 'avg_transit_hours', 'shipment_count']
state_performance = state_performance.sort_values(by='avg_transit_hours', ascending=False)

plt.figure(figsize=(12, 7))
bars = plt.bar(state_performance['origin_state'], state_performance['avg_transit_hours'], 
               color='#2ecc71', edgecolor='black', alpha=0.8)
# Add shipment counts on top of bars
for i, (bar, count) in enumerate(zip(bars, state_performance['shipment_count'])):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'n={int(count)}', ha='center', va='bottom', fontsize=9)
plt.title('Average Transit Hours by Origin State', fontsize=16, fontweight='bold')
plt.xlabel('Origin State', fontsize=12)
plt.ylabel('Average Transit Hours', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.savefig('avg_transit_time_by_state.png', dpi=300, bbox_inches='tight')
plt.close()
print("✓ Saved: avg_transit_time_by_state.png")


Creating: Transit Hours by Origin State...
✓ Saved: avg_transit_time_by_state.png


In [9]:
# --- 2.3. Facility Touchpoints Distribution ---
print("Creating: Facility Touchpoints Distribution...")
facility_dist = df_clean['num_facilities_visited'].value_counts().sort_index().reset_index()
facility_dist.columns = ['num_facilities', 'count']

plt.figure(figsize=(12, 7))
bars = plt.bar(facility_dist['num_facilities'], facility_dist['count'], 
               color='#e74c3c', edgecolor='black', alpha=0.8)
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{int(height)}', ha='center', va='bottom', fontsize=10)
plt.title('Distribution of Facility Touchpoints per Shipment', fontsize=16, fontweight='bold')
plt.xlabel('Number of Unique Facilities Visited', fontsize=12)
plt.ylabel('Number of Shipments', fontsize=12)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.savefig('facility_touchpoints_distribution.png', dpi=300, bbox_inches='tight')
plt.close()
print("✓ Saved: facility_touchpoints_distribution.png")

Creating: Facility Touchpoints Distribution...
✓ Saved: facility_touchpoints_distribution.png


In [11]:
# --- 2.4. Express vs Standard Service Performance ---
print("Creating: Service Type Comparison...")
if 'is_express_service' in df_clean.columns:
    service_comparison = df_clean.groupby('is_express_service').agg({
        'total_transit_hours': ['mean', 'median'],
        'num_facilities_visited': 'mean',
        'first_attempt_delivery': lambda x: (x.sum() / len(x)) * 100,
        'tracking_number': 'count'
    }).round(2)
    
    # Check if we have both service types
    has_both = (False in service_comparison.index and True in service_comparison.index)
    
    if has_both:
        fig, axes = plt.subplots(1, 2, figsize=(14, 6))
        
        # Transit time comparison
        service_labels = ['Standard', 'Express']
        transit_means = [service_comparison.loc[False, ('total_transit_hours', 'mean')],
                         service_comparison.loc[True, ('total_transit_hours', 'mean')]]
        
        axes[0].bar(service_labels, transit_means, color=['#95a5a6', '#f39c12'], 
                    edgecolor='black', alpha=0.8)
        axes[0].set_title('Avg Transit Time by Service Type', fontsize=13, fontweight='bold')
        axes[0].set_ylabel('Average Transit Hours', fontsize=11)
        axes[0].grid(True, alpha=0.3, axis='y')
        
        # First attempt delivery comparison
        first_attempt_pct = [service_comparison.loc[False, ('first_attempt_delivery', '<lambda>')],
                            service_comparison.loc[True, ('first_attempt_delivery', '<lambda>')]]
        
        axes[1].bar(service_labels, first_attempt_pct, color=['#95a5a6', '#f39c12'], 
                    edgecolor='black', alpha=0.8)
        axes[1].set_title('First Attempt Delivery Rate', fontsize=13, fontweight='bold')
        axes[1].set_ylabel('Success Rate (%)', fontsize=11)
        axes[1].set_ylim([0, 100])
        axes[1].grid(True, alpha=0.3, axis='y')
        
        plt.tight_layout()
        plt.savefig('service_type_comparison.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ Saved: service_type_comparison.png")
    else:
        print("⚠ Skipped: Only one service type found in dataset")


Creating: Service Type Comparison...
⚠ Skipped: Only one service type found in dataset


In [12]:
# --- 2.5. Transit Time vs Facilities Scatter Plot ---
print("Creating: Transit Time vs Facilities Correlation...")
plt.figure(figsize=(12, 7))
scatter = plt.scatter(df_clean['num_facilities_visited'], 
                     df_clean['total_transit_hours'],
                     c=df_clean['package_weight_kg'], 
                     cmap='YlOrRd', 
                     alpha=0.6, 
                     s=100,
                     edgecolors='black',
                     linewidth=0.5)
plt.colorbar(scatter, label='Package Weight (kg)')
plt.title('Transit Time vs Number of Facilities Visited', fontsize=16, fontweight='bold')
plt.xlabel('Number of Facilities Visited', fontsize=12)
plt.ylabel('Total Transit Hours', fontsize=12)
plt.grid(True, alpha=0.3)

# Add correlation coefficient
correlation = df_clean[['num_facilities_visited', 'total_transit_hours']].corr().iloc[0, 1]
plt.text(0.05, 0.95, f'Correlation: {correlation:.3f}', 
         transform=plt.gca().transAxes, 
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8),
         fontsize=11, verticalalignment='top')

plt.tight_layout()
plt.savefig('transit_time_vs_facilities.png', dpi=300, bbox_inches='tight')
plt.close()
print("✓ Saved: transit_time_vs_facilities.png")


Creating: Transit Time vs Facilities Correlation...
✓ Saved: transit_time_vs_facilities.png


In [14]:
# --- 2.6. Delivery Performance Metrics ---
print("Creating: Delivery Performance Dashboard...")
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# First attempt delivery rate
first_attempt_rate = (df_clean['first_attempt_delivery'].sum() / len(df_clean)) * 100
axes[0, 0].bar(['First Attempt', 'Multiple Attempts'], 
               [first_attempt_rate, 100 - first_attempt_rate],
               color=['#27ae60', '#e67e22'], edgecolor='black', alpha=0.8)
axes[0, 0].set_title('Delivery Success Rate', fontsize=13, fontweight='bold')
axes[0, 0].set_ylabel('Percentage (%)', fontsize=11)
axes[0, 0].set_ylim([0, 100])
axes[0, 0].grid(True, alpha=0.3, axis='y')

# Out for delivery attempts distribution
ofd_dist = df_clean['num_out_for_delivery_attempts'].value_counts().sort_index()
axes[0, 1].bar(ofd_dist.index, ofd_dist.values, color='#9b59b6', 
               edgecolor='black', alpha=0.8)
axes[0, 1].set_title('Out-for-Delivery Attempts', fontsize=13, fontweight='bold')
axes[0, 1].set_xlabel('Number of Attempts', fontsize=11)
axes[0, 1].set_ylabel('Number of Shipments', fontsize=11)
axes[0, 1].grid(True, alpha=0.3, axis='y')

# Package weight distribution
axes[1, 0].hist(df_clean['package_weight_kg'], bins=20, color='#16a085', 
                edgecolor='black', alpha=0.8)
axes[1, 0].set_title('Package Weight Distribution', fontsize=13, fontweight='bold')
axes[1, 0].set_xlabel('Weight (kg)', fontsize=11)
axes[1, 0].set_ylabel('Number of Shipments', fontsize=11)
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Transit efficiency (hours per facility)
axes[1, 1].hist(df_clean['avg_hours_per_facility'], bins=20, color='#c0392b', 
                edgecolor='black', alpha=0.8)
axes[1, 1].set_title('Transit Efficiency (Hours per Facility)', fontsize=13, fontweight='bold')
axes[1, 1].set_xlabel('Average Hours per Facility', fontsize=11)
axes[1, 1].set_ylabel('Number of Shipments', fontsize=11)
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('delivery_performance_dashboard.png', dpi=300, bbox_inches='tight')
plt.close()
print("✓ Saved: delivery_performance_dashboard.png")

print()



Creating: Delivery Performance Dashboard...
✓ Saved: delivery_performance_dashboard.png



In [26]:
# 3. ADVANCED ANALYSIS: OUTLIER DETECTION
print("ADVANCED ANALYSIS: PERFORMANCE OUTLIERS")
print()

# --- 3.1. Top 5 Worst Performers (Longest Transit Time) ---
print(" TOP 5 WORST PERFORMERS (Longest Transit Time):")
worst_performers = df_clean.nlargest(5, 'total_transit_hours')
worst_summary = worst_performers[[
    'tracking_number', 
    'origin_city', 
    'destination_city', 
    'total_transit_hours', 
    'num_facilities_visited',
    'first_attempt_delivery',
    'service_type'
]]
print(worst_summary.to_string(index=False))
print()

ADVANCED ANALYSIS: PERFORMANCE OUTLIERS

 TOP 5 WORST PERFORMERS (Longest Transit Time):
 tracking_number origin_city  destination_city  total_transit_hours  num_facilities_visited  first_attempt_delivery        service_type
    391198356290   Bangalore         Thanjavur               545.53                       3                    True FEDEX_EXPRESS_SAVER
    390870220230   Bangalore         Hyderabad               223.98                       3                    True FEDEX_EXPRESS_SAVER
    281095533884         Goa           Chennai               223.80                       2                    True FEDEX_EXPRESS_SAVER
    390767871261   Bangalore          Gurugram               222.04                       2                    True FEDEX_EXPRESS_SAVER
    390807999654   Bangalore Bokaro steel City               216.39                       2                    True FEDEX_EXPRESS_SAVER



In [27]:
# --- 3.2. Top 5 Best Performers (Shortest Transit Time) ---
print(" TOP 5 BEST PERFORMERS (Shortest Transit Time):")
best_performers = df_clean.nsmallest(5, 'total_transit_hours')
best_summary = best_performers[[
    'tracking_number', 
    'origin_city', 
    'destination_city', 
    'total_transit_hours', 
    'num_facilities_visited',
    'first_attempt_delivery',
    'service_type'
]]
print(best_summary.to_string(index=False))
print()


 TOP 5 BEST PERFORMERS (Shortest Transit Time):
 tracking_number origin_city destination_city  total_transit_hours  num_facilities_visited  first_attempt_delivery        service_type
    390871801797   Bangalore        Bengaluru                20.60                       3                    True FEDEX_EXPRESS_SAVER
    390950569057   Bangalore        Bengaluru                21.02                       3                    True FEDEX_EXPRESS_SAVER
    390902329207   Bangalore        Bengaluru                22.60                       3                    True FEDEX_EXPRESS_SAVER
    390902302347   Bangalore        Bangalore                22.64                       3                    True FEDEX_EXPRESS_SAVER
    390871742485   Bangalore           Mysore                24.10                       3                    True FEDEX_EXPRESS_SAVER



In [28]:
# --- 3.3. Statistical Outliers (Using IQR Method) ---
print(" STATISTICAL OUTLIERS (IQR Method):")
Q1 = df_clean['total_transit_hours'].quantile(0.25)
Q3 = df_clean['total_transit_hours'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df_clean[(df_clean['total_transit_hours'] < lower_bound) | 
                    (df_clean['total_transit_hours'] > upper_bound)]

print(f"Total outliers detected: {len(outliers)} ({len(outliers)/len(df_clean)*100:.1f}%)")
print(f"Lower bound: {lower_bound:.2f} hours")
print(f"Upper bound: {upper_bound:.2f} hours")
print()

if len(outliers) > 0:
    print("Sample of outlier shipments:")
    print(outliers[['tracking_number', 'total_transit_hours', 'num_facilities_visited', 
                    'origin_city', 'destination_city']].head(10).to_string(index=False))
print()


 STATISTICAL OUTLIERS (IQR Method):
Total outliers detected: 5 (5.1%)
Lower bound: -9.35 hours
Upper bound: 203.59 hours

Sample of outlier shipments:
 tracking_number  total_transit_hours  num_facilities_visited origin_city  destination_city
    390807999654               216.39                       2   Bangalore Bokaro steel City
    390767871261               222.04                       2   Bangalore          Gurugram
    390870220230               223.98                       3   Bangalore         Hyderabad
    391198356290               545.53                       3   Bangalore         Thanjavur
    281095533884               223.80                       2         Goa           Chennai



In [29]:
# --- 3.4. Route-Specific Performance ---
print("  ROUTE-SPECIFIC PERFORMANCE:")
df_clean['route'] = df_clean['origin_city'] + ' → ' + df_clean['destination_city']
route_performance = df_clean.groupby('route').agg({
    'total_transit_hours': ['mean', 'count'],
    'first_attempt_delivery': lambda x: (x.sum() / len(x)) * 100
}).round(2)
route_performance.columns = ['avg_transit_hours', 'shipment_count', 'first_attempt_pct']
route_performance = route_performance[route_performance['shipment_count'] >= 2]
route_performance = route_performance.sort_values('avg_transit_hours', ascending=False)

print("\nTop 5 Slowest Routes (with 2+ shipments):")
print(route_performance.head().to_string())
print()

  ROUTE-SPECIFIC PERFORMANCE:

Top 5 Slowest Routes (with 2+ shipments):
                       avg_transit_hours  shipment_count  first_attempt_pct
route                                                                      
Bangalore → Lucknow               172.69               2             100.00
Bangalore → Kolkata               159.86               2             100.00
Bangalore → Gurugram              158.52               2             100.00
Bangalore → Delhi                 116.07               7              85.71
Bangalore → New Delhi             106.76               5              80.00



In [31]:
# 4. SUMMARY STATISTICS

print("KEY INSIGHTS & SUMMARY")
print()

print(f"Total Shipments Analyzed: {len(df_clean)}")
print(f"Average Transit Time: {df_clean['total_transit_hours'].mean():.2f} hours")
print(f"Median Transit Time: {df_clean['total_transit_hours'].median():.2f} hours")
print(f"Average Facilities per Shipment: {df_clean['num_facilities_visited'].mean():.2f}")
print(f"First Attempt Delivery Rate: {(df_clean['first_attempt_delivery'].sum()/len(df_clean)*100):.1f}%")
print(f"Express Service Shipments: {df_clean['is_express_service'].sum()} ({df_clean['is_express_service'].sum()/len(df_clean)*100:.1f}%)")
print()

print("Analysis complete! All visualizations saved.")
print()
print("Generated files:")
print("  • transit_time_distribution.png")
print("  • avg_transit_time_by_state.png")
print("  • facility_touchpoints_distribution.png")
print("  • service_type_comparison.png")
print("  • transit_time_vs_facilities.png")
print("  • delivery_performance_dashboard.png")

KEY INSIGHTS & SUMMARY

Total Shipments Analyzed: 99
Average Transit Time: 100.86 hours
Median Transit Time: 97.24 hours
Average Facilities per Shipment: 2.83
First Attempt Delivery Rate: 84.8%
Express Service Shipments: 99 (100.0%)

Analysis complete! All visualizations saved.

Generated files:
  • transit_time_distribution.png
  • avg_transit_time_by_state.png
  • facility_touchpoints_distribution.png
  • service_type_comparison.png
  • transit_time_vs_facilities.png
  • delivery_performance_dashboard.png
